#Features extracting from audio

Create spectogram for each audio file and get features matrix from it

In [2]:
import os
import glob
import scipy.io.wavfile as wav
import numpy as np
import os.path
from PIL import Image
import pickle
os.chdir('/Users/vidmantasbendikas/Documents/github/soundrecognition')
audio_folder = 'audio_preparation/Audio_splitted'
spectrograms_folder = 'audio_preparation/spectrograms'
audios_features_folder = 'audios_features'
LOG_EVERY_N = 100

Preprocessed 100 people records


Preprocessed 200 people records


Preprocessed 300 people records


Preprocessed 400 people records


Preprocessed 500 people records


...

Preprocessed 1000 people records


Preprocessed 1100 people records


Preprocessed 1200 people records


Preprocessed 1300 people records


Preprocessed 1400 people records


In [ ]:
if not os.path.exists(spectrograms_folder):
    os.makedirs(spectrograms_folder)


list_subdirs = os.listdir(audio_folder)
s=0
for i in list_subdirs:
    s += 1
    if (s % LOG_EVERY_N) == 0:
        print "Preprocessed "+str(s)+" people records"  
    directory = audio_folder+'/'+i+'/'
    new_directory = spectrograms_folder+'/'+i+'/'
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)
    #
    list_wav = glob.glob(directory+"*.wav")
    for k in range(len(list_wav)):
        spectrogram_dir = spectrograms_folder+'/'+i+'/'+ i+'_'+list_wav[k].split('/')[-1].split('.')[0]+'.png'
        #command = "sox "+list_wav[k]+" -n spectrogram -l -r -o "+spectrogram_dir
        command = "sox "+list_wav[k]+" -n spectrogram -l -m -r -w Kaiser -o "+spectrogram_dir
        os.system(command)


Transform spectrogram image into matrix and save it to tuple

In [3]:
def img_to_matrix(filename, verbose=False, STANDARD_SIZE= (120,120)):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    if verbose==True:
        print "changing size from %s to %s" % (str(img.size), str(STANDARD_SIZE))
    img = img.resize(STANDARD_SIZE)
    img = list(img.getdata())
    img = np.array(img)
    return img

spectograms_subfolders = os.listdir(spectrograms_folder)
audios_features = {}
s = 0
for subfolder in spectograms_subfolders:
    s += 1
    if (s % LOG_EVERY_N) == 0:
        print "Transformed "+str(s)+" spectrograms into array"  
    path_to_spectograms_files = spectrograms_folder+'/'+subfolder+'/'
    spectrograms_files_list = glob.glob(path_to_spectograms_files+"*.png")
    
    audios_features[subfolder] = {}
    audios_features[subfolder]['device'] = {}
    audios_features[subfolder]['device']['studio'] = {}
    audios_features[subfolder]['device']['iphone'] = {}
    audios_features[subfolder]['device']['samsung'] = {}
    for spectrogram_file in spectrograms_files_list:
        if spectrogram_file.split('_')[-1].split('a')[0]=='s':
            audio_features = img_to_matrix(spectrogram_file)
            audios_features[subfolder]['device']['samsung'][spectrogram_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['samsung'][spectrogram_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['samsung'][spectrogram_file.split('/')[-1]]['features']['spectrogram'] = audio_features
       
        elif spectrogram_file.split('_')[-1].split('a')[0]=='i':
            audio_features = img_to_matrix(spectrogram_file)
            audios_features[subfolder]['device']['iphone'][spectrogram_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['iphone'][spectrogram_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['iphone'][spectrogram_file.split('/')[-1]]['features']['spectrogram'] = audio_features 
        else:
            audio_features = img_to_matrix(spectrogram_file)
            audios_features[subfolder]['device']['studio'][spectrogram_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['studio'][spectrogram_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['studio'][spectrogram_file.split('/')[-1]]['features']['spectrogram'] = audio_features

if not os.path.exists(audios_features_folder):
    os.makedirs(audios_features_folder)

pickle.dump(audios_features, open(audios_features_folder+"/"+"audios_features.p", "wb" ))

Transformed 100 spectrograms into array


Transformed 200 spectrograms into array


Transformed 300 spectrograms into array


Transformed 400 spectrograms into array


Transformed 500 spectrograms into array


...

Transformed 1000 spectrograms into array


Transformed 1100 spectrograms into array


Transformed 1200 spectrograms into array


Transformed 1300 spectrograms into array


Transformed 1400 spectrograms into array


Extract features from audio using soundnet

In [ ]:
command = "python extract_feat.py -m 24 -x 25 -s -p extract -t data.txt"
os.system(command)

The power spectrum of each segment can be obtained by applying the Fourier transform, which converts the waveform data from the time domain to the frequency domain. The code below demonstrates how to use NumPy’s Fourier transform module.

In [ ]:
def split_recording(recording, segment_length, sample_rate):
    segments = []
    index = 0
    while index < len(recording):
        segment = recording[index:int(index + segment_length*sample_rate)]
        segments.append(segment)
        index += segment_length*sample_rate
    return segments

def calculate_normalized_power_spectrum(recording, sample_rate):
    # np.fft.fft returns the discrete fourier transform of the recording
    fft = np.fft.fft(recording)
    number_of_samples = len(recording)
    # sample_length is the length of each sample in seconds
    sample_length = 1./sample_rate
    # fftfreq is a convenience function which returns the list of frequencies measured by the fft
    frequencies = np.fft.fftfreq(number_of_samples, sample_length)
    positive_frequency_indices = np.where(frequencies>0)
    # positive frequences returned by the fft
    frequencies = frequencies[positive_frequency_indices]
    # magnitudes of each positive frequency in the recording
    magnitudes = abs(fft[positive_frequency_indices])
    # some segments are louder than others, so normalize each segment
    magnitudes = magnitudes / np.linalg.norm(magnitudes)
    return frequencies, magnitudes

def create_power_spectra_array(segment_length, sample_rate):
    number_of_samples_per_segment = int(segment_length * sample_rate)
    time_per_sample = 1./sample_rate
    frequencies = np.fft.fftfreq(number_of_samples_per_segment, time_per_sample)
    positive_frequencies = frequencies[frequencies>0]
    power_spectra_array = np.empty((0, len(positive_frequencies)))
    return power_spectra_array

def fill_power_spectra_array(splits, power_spectra_array, fs):
    filled_array = power_spectra_array
    for segment in splits:
        freqs, mags = calculate_normalized_power_spectrum(segment, fs)
        filled_array = np.vstack((filled_array, mags))
    return filled_array


# wav.read returns the sample_rate and a numpy array containing each audio sample from the .wav file
#sample_rate, recording = wav.read(filename)
segment_length = 1 # length in seconds
#segments = split_recording(recording, segment_length, sample_rate)
#power_spectra_array = create_power_spectra_array(segment_length,sample_rate)
#power_spectra_array = fill_power_spectra_array(segments, power_spectra_array, sample_rate)

list_subfolders = os.listdir(audio_folder)
audios_features = {}
s = 0
for subfolder in list_subfolders:
    s += 1
    if (s % LOG_EVERY_N) == 0:
        print "Transformed "+str(s)+" audio into power spectrum array"  
    path_to_wav_files = audio_folder+'/'+subfolder+'/'
    wav_files_list = glob.glob(path_to_wav_files+"*.wav")
    
    audios_features[subfolder] = {}
    audios_features[subfolder]['device'] = {}
    audios_features[subfolder]['device']['studio'] = {}
    audios_features[subfolder]['device']['iphone'] = {}
    audios_features[subfolder]['device']['samsung'] = {}
    for wav_file in wav_files_list:
        sample_rate, recording = wav.read(wav_file)
        segments = split_recording(recording, segment_length, sample_rate)
        audio_features = create_power_spectra_array(segment_length,sample_rate)
        audio_features = fill_power_spectra_array(segments, audio_features, sample_rate)
        
        if wav_file.split('_')[-1].split('a')[0]=='s':
            audios_features[subfolder]['device']['samsung'][wav_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['samsung'][wav_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['samsung'][wav_file.split('/')[-1]]['features']['power_spectrum'] = audio_features[0]
       
        elif wav_file.split('_')[-1].split('a')[0]=='i':
            audios_features[subfolder]['device']['iphone'][wav_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['iphone'][wav_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['iphone'][wav_file.split('/')[-1]]['features']['power_spectrum'] = audio_features[0] 
        else:
            audios_features[subfolder]['device']['studio'][wav_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['studio'][wav_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['studio'][wav_file.split('/')[-1]]['features']['power_spectrum'] = audio_features[0]

if not os.path.exists(audios_features_folder):
    os.makedirs(audios_features_folder)

pickle.dump(audios_features, open(audios_features_folder+"/"+"audios_features_power_spectrum.p", "wb" ))